In [ ]:
import pandas as pd

from common_utils.utils import get_json_obj
from haversine import haversine, Unit
from random import randint

In [ ]:
airports_geometry = get_json_obj("/home/magnus9102/Mostafa/Py/Github/data-science/mostafa_vahdani_bachelor_project/source_codes/data/static/airports_geometry.json")

In [ ]:
df = pd.read_csv("/home/magnus9102/Mostafa/Py/Github/data-science/mostafa_vahdani_bachelor_project/data/raw/flight_tickets_dataset.csv")

In [ ]:
airport_name1, coord1 = airports_geometry["MHD"]["aeroway_name"], airports_geometry["MHD"]["geometry"]


In [ ]:
airport_name2, coord2 = airports_geometry["THR"]["aeroway_name"], airports_geometry["THR"]["geometry"]


In [ ]:
coord1, coord2

In [ ]:
airport_name1, airport_name2

In [ ]:
def difference_drop(df, *args):
    """difference drop with column names that you will give on *args"""
    return df.drop(columns=df.columns.difference([*args]), axis=1)


In [ ]:
df2 = difference_drop(df, "national_departure_code", "national_arrival_code")


In [ ]:
def orthodromic_distance(row, airports_geometry_dict):
    departure_airport_code = row["national_departure_code"]
    arrival_airport_code = row["national_arrival_code"]

    departure_airport_coordinate = airports_geometry_dict[departure_airport_code]["geometry"]
    arrival_airport_coordinate = airports_geometry_dict[arrival_airport_code]["geometry"]

    return round(haversine(departure_airport_coordinate, 
                     arrival_airport_coordinate, 
                     Unit.KILOMETERS, 
                     normalize=True))


df["orthodromic_distance_KM"] = df2.apply(func=orthodromic_distance,
                                           args=(airports_geometry,),
                                           axis=1)


In [ ]:
df["orthodromic_distance_KM"].head(5)

In [ ]:
def estimate_flight_length(distance):
    # per minutes
    v_km_per_min = randint(885, 965) / 60
    taxi_time = randint(5, 10)
    attach_stairs_time = randint(5, 10)
    takeoff_clearance = randint(5, 10)
    takeoff_time = randint(10, 20)
    cruise_time = distance / v_km_per_min
    vectoring_time = 10
    busy_airplane_time = 10
    descent_time = randint(10, 15)
    landing_time = randint(10, 20)

    flight_length = descent_time + landing_time + takeoff_time + taxi_time + attach_stairs_time + busy_airplane_time + vectoring_time + cruise_time + takeoff_clearance

    return round(flight_length)


In [ ]:
df2 = difference_drop(df, "orthodromic_distance_KM")


In [ ]:
df["flight_length"] = df2.apply(func=estimate_flight_length, axis=1)


In [ ]:
df["flight_length"].head(5)
